In [8]:
import pandas as pd
import csv

csv_filepath = "constituency_socioeconomic_backgrounds.csv"
data_list = []

# read in data
with open(csv_filepath, newline='', encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile)
    
    for row in csv_reader:
        new_row = [row[1], row[6], row[7]]
        data_list.append(new_row)

data_list.remove(data_list[0])


# specifying column names
df = pd.DataFrame(data_list, columns=['Constituency', 'Class', 'Count'])
df['Count'] = pd.to_numeric(df['Count'], errors='coerce')

#print (df[:10])

# this organises the dataframe where each class is a collumn
grouped_df = df.groupby(['Constituency', 'Class']).sum().reset_index()
pivot_df = grouped_df.pivot(index='Constituency', columns='Class', values='Count')

#print (pivot_df[:10])

class_rename_dict = {
    'Managerial, administrative and professional occupations': 'Professional',
    'Intermediate occupations': 'Intermediate',
    'Routine and manual occupations': 'Manual',  
}

pivot_df = pivot_df.rename(columns=class_rename_dict)

pivot_df = pivot_df.reset_index()

# this calculates the total relevant population for each constituency
pivot_df["Total_Count"] = pivot_df["Professional"] + pivot_df["Intermediate"] + pivot_df["Manual"]

# this takes class percentages and puts them each in a seperate column with _perc added to the column title
perc_list = []
for class_short in class_rename_dict.values():
    perc_column = f"{class_short}_Perc"
    pivot_df[perc_column] = (pivot_df[class_short] / pivot_df["Total_Count"]) * 100
    perc_list.append(perc_column)

#print (perc_list)

# this then just takes the mean and standard deviation of each perc column and puts them in a seperate dataframe
mean_df = pivot_df[perc_list].mean()
std_df = pivot_df[perc_list].std()
stats_df = pd.DataFrame({'Mean': mean_df, 'Standard Deviation': std_df})

print(stats_df)










                        Mean  Standard Deviation
Class                                           
Professional_Perc  38.906691            8.338382
Intermediate_Perc  26.200713            2.678340
Manual_Perc        34.892597            8.544008
